In [ ]:
import pandas as pd
import numpy as np
import csv
import re

np.random.seed(500)

In [ ]:
# Data extraction configurations
drug_occur_threshold = 20000

drug_unknown = 'UNKNOWN'

ndc_code_filename = '../data/processed/ndc_codes_extracted.csv'

### Threshold, Result Count
#   1000, 2229
#    500, 2957
#    200, 4005
#    100, 4777
#     50, 5624
#     10, 7511
#     5, 8148
#     2, 8710

In [ ]:
df = pd.read_csv('../data/real-mimic-iii-database/PRESCRIPTIONS.csv', dtype=str)

In [ ]:
df = df[(df['NDC'] != "0") & (df['NDC'] != None)]

In [ ]:
df['DRUG_NORMALIZED'] = df['DRUG'].apply(lambda x: re.sub('[^a-z ]+', '', x.lower()).strip())
df = df[df['DRUG_NORMALIZED'].str.len() >= 10]

In [ ]:
df_ndc_freq = df[['ROW_ID', 'DRUG_NORMALIZED']].groupby('DRUG_NORMALIZED').count().sort_values('ROW_ID', ascending=False)
df_ndc_freq = df_ndc_freq.reset_index()
df_ndc_freq.head(20)

In [ ]:
df_ndc_most_freq_drugs = df_ndc_freq[(df_ndc_freq['ROW_ID'] >= drug_occur_threshold)]
df_ndc_most_freq_drugs

In [ ]:
most_freq_ndc_drugs = df_ndc_most_freq_drugs['DRUG_NORMALIZED'].to_list()

In [ ]:
df_ndc_most_freq_meta = df[df['DRUG_NORMALIZED'].isin(set(most_freq_ndc_drugs))]
df_ndc_most_freq = df_ndc_most_freq_meta[['NDC','DRUG_NORMALIZED']].groupby('DRUG_NORMALIZED').first().reset_index()
print("length: ", len(df_ndc_most_freq))

In [ ]:
df_ndc_most_freq.to_csv(ndc_code_filename, sep=',', encoding='utf-8', quotechar='"', quoting=csv.QUOTE_ALL, index=False)